In [1]:
import numpy as np
import cv2
import time

In [2]:
driveName = r"\\.\PhysicalDrive1"  # Change this to the correct drive

# SD Card sector information
headerSector =          1023 # Holds user settings to configure Miniscope and recording
configSector =          1024 # Holds final settings of the actual recording
dataStartSector =       1025 # Recording data starts here
sectorSize =            512

WRITE_KEY0 =				0x0D7CBA17
WRITE_KEY1 =				0x0D7CBA17
WRITE_KEY2 =				0x0D7CBA17
WRITE_KEY3 =				0x0D7CBA17

# SD Card Header Sector positions
HEADER_GAIN_POS =				4
HEADER_LED_POS =				5
HEADER_EWL_POS =				6
HEADER_RECORD_LENGTH_POS =  	7
HEADER_FRAME_RATE = 			8

# SD Card Config Sector positions
CONFIG_BLOCK_WIDTH_POS =			    0
CONFIG_BLOCK_HEIGHT_POS	=   		    1
CONFIG_BLOCK_FRAME_RATE_POS	=   	    2
CONFIG_BLOCK_BUFFER_SIZE_POS =  	    3
CONFIG_BLOCK_NUM_BUFFERS_RECORDED_POS =	4
CONFIG_BLOCK_NUM_BUFFERS_DROPPED_POS =	5

# Data Buffer Header positions
BUFFER_HEADER_HEADER_LENGTH_POS =			0
BUFFER_HEADER_LINKED_LIST_POS = 			1
BUFFER_HEADER_FRAME_NUM_POS	=   			2
BUFFER_HEADER_BUFFER_COUNT_POS =			3
BUFFER_HEADER_FRAME_BUFFER_COUNT_POS =  	4
BUFFER_HEADER_WRITE_BUFFER_COUNT_POS =  	5
BUFFER_HEADER_DROPPED_BUFFER_COUNT_POS =	6
BUFFER_HEADER_TIMESTAMP_POS	=   			7
BUFFER_HEADER_DATA_LENGTH_POS = 			8

In [10]:
# variables that you can adjust
correctDrive = False
f = open(driveName, "rb")  # Open drive

# Make sure this is the correct drive
# Read SD Card header and config sectors
f.seek(headerSector * sectorSize, 0)  # Move to correct sector
headerSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)
if ((WRITE_KEY0 == headerSectorData[0]) and (WRITE_KEY1 == headerSectorData[1]) and (WRITE_KEY2 == headerSectorData[2]) and (WRITE_KEY3 == headerSectorData[3])):
    correctDrive = True
    print("SD Card Openned.")
else:
    print ("Wrong Drive.")
    correctDrive = False
    f.close()

SD Card Openned.


In [9]:
f.close()

In [11]:
# Load up Config Sector
f.seek(configSector * sectorSize, 0)  # Move to correct sector
configSectorData = np.fromstring(f.read(sectorSize), dtype=np.uint32)
configSectorData

array([  608,   608,    20, 64000,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [12]:
# Read Data Sectors
frameNum = 0
pixelCount = 0

frame = np.empty((configSectorData[CONFIG_BLOCK_WIDTH_POS] * configSectorData[CONFIG_BLOCK_HEIGHT_POS], 1), dtype=np.uint8)
f.seek(dataStartSector * sectorSize, 0) # Starting data location
for i in range(5000):
    dataHeader = np.fromstring(f.read(4), dtype=np.uint32)
    dataHeader = np.append(dataHeader, np.fromstring(f.read((dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] - 1) * 4), dtype=np.uint32))

    print(dataHeader)
    numBlocks = int((dataHeader[BUFFER_HEADER_DATA_LENGTH_POS] + (dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4) + (512 - 1)) / 512)
    data = np.fromstring(f.read(numBlocks*512 - dataHeader[BUFFER_HEADER_HEADER_LENGTH_POS] * 4), dtype=np.uint8)

    # -------------------------------------
    if (dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] == 0):
        # First buffer of a frame
        frame[0:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS], 0] = data
        pixelCount = dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        frameNum = dataHeader[BUFFER_HEADER_FRAME_NUM_POS]
    else:
        # All other buffers of a frame
        if (dataHeader[BUFFER_HEADER_FRAME_BUFFER_COUNT_POS] < 5):
            frame[pixelCount:(pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]), 0] = data
            pixelCount = pixelCount + dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]
        else:
            # Last buffer of frame. This should be handled better once amount of data is correctly added to header
            frame[pixelCount:, 0] = data[:dataHeader[BUFFER_HEADER_DATA_LENGTH_POS]]
            cv2.imshow('Video', np.reshape(frame, (configSectorData[CONFIG_BLOCK_WIDTH_POS], configSectorData[CONFIG_BLOCK_HEIGHT_POS] )))
            cv2.waitKey(50)
        
cv2.destroyWindow('Video')


[    9     0     0     0     0     0     0    30 63964]
[    9     1     0     1     1     1     0    34 63964]
[    9     2     0     2     2     2     0    39 63964]
[    9     3     0     3     3     3     0    43 63964]
[    9     4     0     4     4     4     0    47 63964]
[    9     0     0     5     5     5     0    51 49844]
[    9     1     1     6     0     6     0    80 63964]
[    9     2     1     7     1     7     0    85 63964]
[    9     3     1     8     2     8     0    89 63964]
[    9     4     1     9     3     9     0    93 63964]
[    9     0     1    10     4    10     0    98 63964]
[    9     1     1    11     5    11     0   101 49844]
[    9     2     2    12     0    12     0   131 63964]
[    9     3     2    13     1    13     0   135 63964]
[    9     4     2    14     2    14     0   140 63964]
[    9     0     2    15     3    15     0   144 63964]
[    9     1     2    16     4    16     0   148 63964]
[    9     2     2    17     5    17     0   152

ValueError: read length must be non-negative or -1

In [122]:
f.close()

In [13]:
f.seek(10024 * 512)
count = 10024
for i in range(4000):
    temp = np.fromstring(f.read(512), dtype=np.uint32)
    
    if (temp[0] == 7 or temp[0] == 9):
        print(count)
        print(temp[:9])
    count = count + 1


10076
[    9     0    12    75     3    75     0   650 63964]
10201
[    9     1    12    76     4    76     0   654 63964]
10326
[    9     2    12    77     5    77     0   658 49844]
10424
[    9     3    13    78     0    78     0   688 63964]
10549
[    9     4    13    79     1    79     0   692 63964]
10674
[    9     0    13    80     2    80     0   696 63964]
10799
[    9     1    13    81     3    81     0   701 63964]
10924
[    9     2    13    82     4    82     0   705 63964]
11049
[    9     3    13    83     5    83     0   708 49844]
11147
[    9     4    14    84     0    84     0   738 63964]
11272
[    9     0    14    85     1    85     0   743 63964]
11397
[    9     1    14    86     2    86     0   747 63964]
11522
[    9     2    14    87     3    87     0   751 63964]
11647
[    9     3    14    88     4    88     0   756 63964]
11772
[    9     4    14    89     5    89     0   759 49844]
11870
[    9     0    15    90     0    90     0   789 63964]
11995
[ 

In [ ]:
# Delete data from SD Card
f.seek(dataStartSector * sectorSize, 0)

zeros = []
for i in range(512):
    zeros.append(0)
binaryZeros = bytearray(zeros)
f.write(binaryZeros)